In [22]:
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
from tensorflow import keras
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential, layers
import os
from tensorflow.keras.applications import Xception
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import itertools
from sklearn.preprocessing import MinMaxScaler
from cv2 import blur

dir_ = '../input/150k-mars-data/images/training/dunes'
clean_file_names = [x for x in os.listdir(dir_) if x.endswith('.jpg')]

wind_strength = []
sin = []
cos = []


for i,k,j in itertools.zip_longest(clean_file_names, sin, cos):
    wind_strength.append(i.split('_')[6].strip('.jpg'))
    sin.append(i.split('_')[5].strip('.jpg'))
    cos.append(i.split('_')[4].strip('.jpg'))
    
df = pd.DataFrame()

df['filename'],  df['wind_strength'], df['sin'], df['cos']=\
                                    [clean_file_names, wind_strength, sin, cos]

df['wind_strength'] = df['wind_strength'].apply(float)
df['sin'] = df['sin'].apply(float)
df['cos'] = df['cos'].apply(float)

scaler = MinMaxScaler()

df['wind_strength'] = scaler.fit_transform(df[['wind_strength']])

def blur_func(img, k=5):
    ksize = (k,k)
    blured = blur(img,ksize)
    return blured

data_gen = ImageDataGenerator(rescale=1./255,
                                    brightness_range = [0.5,1.5],
                             preprocessing_function = blur_func)
data_flow = data_gen.flow_from_dataframe(dataframe=df,
                                                    directory=dir_,
                                                    x_col="filename",
                                                    y_col=["wind_strength", 'sin', 'cos'],
                                                    batch_size=16,
                                                    class_mode= 'raw',
                                                    save_format='png',
                                                    color_mode="rgb",
                                                    target_size=(256, 256))



val_dir = '../input/150k-mars-data/images/validation/dunes'
clean_file_names_val = [x for x in os.listdir(val_dir) if x.endswith('.jpg')]
wind_strength_val = []
sin_val = []
cos_val = []

for i,k,j in itertools.zip_longest(clean_file_names_val, sin_val, cos_val):
    wind_strength_val.append(i.split('_')[6].strip('.jpg'))
    sin_val.append(i.split('_')[5].strip('.jpg'))
    cos_val.append(i.split('_')[4].strip('.jpg'))

df_val = pd.DataFrame()

df_val['filename'],  df_val['wind_strength'], df_val['sin'], df_val['cos']= \
                                            [clean_file_names_val, wind_strength_val, sin_val, cos_val]
df_val['wind_strength'] = df_val['wind_strength'].apply(float)
df_val['sin'] = df_val['sin'].apply(float)
df_val['cos'] = df_val['cos'].apply(float)

df_val['wind_strength'] = scaler.transform(df_val[['wind_strength']])

val_gen = ImageDataGenerator(rescale=1./255)
val_flow = val_gen.flow_from_dataframe(dataframe=df_val,
                                        directory=val_dir,
                                        x_col="filename",
                                        y_col=["wind_strength", 'sin', 'cos'],
                                        batch_size=16,
                                        class_mode= 'raw',
                                        color_mode="rgb",
                                        target_size=(256, 256))

def load_model():
    model = Xception(
                        include_top=False, weights= 'imagenet',
                        input_shape=(256, 256,3), pooling="max",
    )
    model.trainable = False 
    model = keras.models.Sequential([ 
                            model,
                            keras.layers.Dense(1020, activation = 'relu'),
                            keras.layers.Dropout(0.3),
                            keras.layers.Dense(200, activation = 'relu'),
                            keras.layers.Dense(3, activation='linear')
                                ])
    opt = Adam(learning_rate=0.001)
    model.compile(loss='mse',
                  optimizer=opt,
                  metrics=['mse', 'mae'])
    return model
mc2 = ModelCheckpoint('Xception_150kmars_best_weight.h5', save_best_only=True)
es = EarlyStopping(monitor = 'val_mse', mode = 'min', patience = 20, restore_best_weights=True)


model = load_model()
history = model.fit(data_flow, epochs=600, verbose = True, callbacks = [es, mc2], validation_data = val_flow)
mse = model.evaluate(val_flow)
pd.DataFrame(history.history)[["mse", "val_mse"]].plot()
pd.DataFrame(history.history)[["mae", "val_mae"]].plot()
model.save(f"Xception_150kmars_{mse}.h5")

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
from tensorflow import keras
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential, layers
import os
from tensorflow.keras.applications import InceptionResNetV2
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import itertools
from sklearn.preprocessing import MinMaxScaler
from cv2 import blur

dir_ = '../input/150k-mars-data/images/training/dunes'
clean_file_names = [x for x in os.listdir(dir_) if x.endswith('.jpg')]

wind_strength = []
sin = []
cos = []


for i,k,j in itertools.zip_longest(clean_file_names, sin, cos):
    wind_strength.append(i.split('_')[6].strip('.jpg'))
    sin.append(i.split('_')[5].strip('.jpg'))
    cos.append(i.split('_')[4].strip('.jpg'))
    
df = pd.DataFrame()

df['filename'],  df['wind_strength'], df['sin'], df['cos']=\
                                    [clean_file_names, wind_strength, sin, cos]

df['wind_strength'] = df['wind_strength'].apply(float)
df['sin'] = df['sin'].apply(float)
df['cos'] = df['cos'].apply(float)

scaler = MinMaxScaler()

df['wind_strength'] = scaler.fit_transform(df[['wind_strength']])

def blur_func(img, k=5):
    ksize = (k,k)
    blured = np.expand_dims(blur(img,ksize), axis = 2)
    return blured

data_gen = ImageDataGenerator(rescale=1./255,
                                    brightness_range = [0.5,1.5],
                             preprocessing_function = blur_func)
data_flow = data_gen.flow_from_dataframe(dataframe=df,
                                                    directory=dir_,
                                                    x_col="filename",
                                                    y_col=["wind_strength", 'sin', 'cos'],
                                                    batch_size=16,
                                                    class_mode= 'raw',
                                                    save_format='png',
                                                    color_mode="grayscale",
                                                    target_size=(256, 256))



val_dir = '../input/150k-mars-data/images/validation/dunes'
clean_file_names_val = [x for x in os.listdir(val_dir) if x.endswith('.jpg')]
wind_strength_val = []
sin_val = []
cos_val = []

for i,k,j in itertools.zip_longest(clean_file_names_val, sin_val, cos_val):
    wind_strength_val.append(i.split('_')[6].strip('.jpg'))
    sin_val.append(i.split('_')[5].strip('.jpg'))
    cos_val.append(i.split('_')[4].strip('.jpg'))

df_val = pd.DataFrame()

df_val['filename'],  df_val['wind_strength'], df_val['sin'], df_val['cos']= \
                                            [clean_file_names_val, wind_strength_val, sin_val, cos_val]
df_val['wind_strength'] = df_val['wind_strength'].apply(float)
df_val['sin'] = df_val['sin'].apply(float)
df_val['cos'] = df_val['cos'].apply(float)

df_val['wind_strength'] = scaler.transform(df_val[['wind_strength']])

val_gen = ImageDataGenerator(rescale=1./255)
val_flow = val_gen.flow_from_dataframe(dataframe=df_val,
                                        directory=val_dir,
                                        x_col="filename",
                                        y_col=["wind_strength", 'sin', 'cos'],
                                        batch_size=16,
                                        class_mode= 'raw',
                                        color_mode="grayscale",
                                        target_size=(256, 256))

def load_model():
    model = Xception(
                        include_top=False, weights=None,
                        input_shape=(256, 256,1), pooling="max",
    )
    model.trainable = True 
    model = keras.models.Sequential([ 
                            model,
                            keras.layers.Dense(3, activation='relu')
                                ])
    opt = Adam(learning_rate=0.001)
    model.compile(loss='mse',
                  optimizer=opt,
                  metrics=['mse', 'mae'])
    return model

mc2 = ModelCheckpoint('InceptionResNetV2_150kmars_best_weight_no_transfer_learning.h5', save_best_only=True)
es = EarlyStopping(monitor = 'val_mse', mode = 'min', patience = 20, restore_best_weights=True)


model = load_model()
history = model.fit(data_flow, epochs=600, verbose = True, callbacks = [es, mc2], validation_data = val_flow)
mse = model.evaluate(val_flow)
pd.DataFrame(history.history)[["mse", "val_mse"]].plot()
pd.DataFrame(history.history)[["mae", "val_mae"]].plot()
model.save(f"InceptionResNetV2_150kmars_{mse}.h5")